# pwy_summary

In [ ]:
import pandas as pd
import cobra
import itertools
from typing import Dict, List
from collections import defaultdict
import ast
import os
import numpy as np

# E0 = cobra.io.read_sbml_model("./models/iML1515_E0.xml")
# # reactions in TAC or other pathways

# desired_cycle = pd.read_csv('./Data/desired_cycle.csv', index_col=0) #flux analysis.ipynb
# gr_DG = pd.read_csv('./r/data fitting/standardized_gr_Div_DG_Mult_Aug31.csv', index_col='gene_inhibition')
# gr_DG.loc[:,:'S0.ac_monoculture'][gr_DG.loc[:,:'S0.ac_monoculture']<1e-5] = 0 

# label_df = pd.read_excel(open('./Data/iML1515_GP.xlsx', 'rb'), # ? S0 rct pathway label necessary?
#               sheet_name='iML1515_GP', index_col=0)
# # rct_pathway_df = pd.read_csv('./Data/rct_pathway.csv')
# alpha_table = pd.read_csv("./Data/alpha_table.csv", index_col=0)
# comb_list = list(pd.read_csv('./Data/gr_Div_DG_Blis_Aug31.csv').gene_inhibition[1:])
DG_list = list(gr_DG.index)
DG_list.remove('Normal')
genes = alpha_table.index

S0 = cobra.io.read_sbml_model("./models/STM_v1_0_S0.xml")
E0.id = 'E0'
S0.id = 'S0.ac'

In [ ]:
def convert_arg_to_list(arg):
    if type(arg) is pd.Series:
        arg = list(arg) 
    elif type(arg) not in [list, tuple, set]:
        arg = [arg] # differ from list(arg) -> conversion of str
    return arg

def get_Biomass_df(files):
    return pd.concat(
            [pd.read_csv(file, index_col='cycle')
             for file in convert_arg_to_list(files)]
        ,axis=1)

Biomass_df = get_Biomass_df(['./Data/BM_SG1.csv', './Data/BM_DG1.csv'])

# desired cycle 


In [ ]:
# pd.set_option('display.max_rows', 2000)
# display(HTML("<style>.container { width:40% !important; }</style>"))

In [ ]:
log_step=5

def search_gr_cycle_with_biomass(df_search, biomass_values):
    return [df_search[df_search >= biomass_value].idxmin() 
                for biomass_value in convert_arg_to_list(biomass_values)]

def get_cycle_max_gr(desired_BM):
    max_gr = desired_BM.iloc[-1]/2
    bool_growing = ((desired_BM.iloc[-1]-desired_BM.iloc[-5])/desired_BM.iloc[-1]).apply(lambda x: x > 1e-10)
    for k, bool_grow in bool_growing.items():
        if bool_grow:
            max_gr[k] = desired_BM[k].iloc[-6]
    biomass_diff = (Biomass_df.iloc[-1]-Biomass_df.iloc[0])
    start = Biomass_df.iloc[0] + biomass_diff*0.1
    end = Biomass_df.iloc[0] + biomass_diff*0.9
    
    return max_gr, start, end, bool_growing

def find_down_neighbour(df):
    max_gr = get_cycle_max_gr(df)
    position = list()
    
    def correct_cycle(cycle):
        mod = cycle%log_step
#         return cycle if mod==0 else cycle+log_step-mod # round up
        return cycle if mod==0 else cycle-mod # round down
#   
    gr_cycle = defaultdict(dict)
    for gcomb, biomass_value in max_gr.items():
        df_search = df.loc[:,gcomb]
        upperneighbour_ind =  search_gr_cycle_with_biomass(df_search, biomass_value) # 0 if growth belolw 1e-8 
#         gr_cycle[gcomb] = correct_cycle(upperneighbour_ind) 
        gr_cycle[gcomb].update({'cycle': correct_cycle(upperneighbour_ind),
                                'growth phase': [search_gr_cycle_with_biomass(),
                                                search_gr_cycle_with_biomass()]})
    return gr_cycle

# def get_desired_cycle_df():
#     desired_cycle_dict = defaultdict(dict)
#     for k, cycle in find_down_neighbour(Biomass_df).items():
#         key_items = k.split('_')
#         gcomb = key_items.pop(1)
#         col_name = '_'.join(key_items)
#         if desired_cycle_dict[gcomb].get(col_name):
#             sys.exit("Halt, Gene comb keys get overwritten")
#         desired_cycle_dict[gcomb].update({col_name: cycle})

#     return pd.DataFrame.from_dict(desired_cycle_dict, orient='index')


In [ ]:
def get_desired_cycle(desired_biomass_df):
    def correct_cycle(cycle):
        mod = cycle%log_step
    #         return cycle if mod==0 else cycle+log_step-mod # round up
        return cycle if mod==0 else cycle-mod # round down

    def get_growth_phase_length():
        return ((desired_cycle['end'] - desired_cycle['start'])*(1-desired_cycle.bool_growing) + # if not growing, not changing growth phase length
                999*(desired_cycle.bool_growing)) #if growing, set growth length to 999

    desired_cycle = (desired_biomass_df.iloc[:-1]
                  .apply(lambda x: 
                         search_gr_cycle_with_biomass(Biomass_df.loc[:,x.name],x))
                  .T)
    def split_index_to_cols(df):
        return pd.DataFrame(zip(*df.index.str.split('_')), index=['Species', 'Gene_inhibition', 'culture'], columns=df.index).T
    desired_cycle['bool_growing'] = desired_biomass_df.T.bool_growing
    desired_cycle['cycle_max_gr'] = desired_cycle['max_gr'].apply(correct_cycle) # -> cycle_max_gr
    desired_cycle['growth_phase'] = desired_cycle[['start', 'end']].values.tolist()
    desired_cycle['growth_phase_length'] = get_growth_phase_length()
    desired_cycle = desired_cycle.join(split_index_to_cols(desired_cycle))
#     .query('culture=="coculture"')
    desired_cycle = desired_cycle.set_index('Gene_inhibition')[['cycle_max_gr', 'bool_growing', 'growth_phase_length', 'Species','culture']]
    
    return desired_cycle

In [ ]:
desired_biomass_df = pd.DataFrame(get_cycle_max_gr(Biomass_df), index=['max_gr', 'start', 'end', 'bool_growing'])
desired_cycle = get_desired_cycle(desired_biomass_df)

# PWY

In [ ]:
def get_gene_id(model, gene_name):
    for i in model.genes:
        if(i.name == gene_name):
            return(i.id)

def record_rct(model, current_gene):
    return [rct.id for rct in model.genes.get_by_id(get_gene_id(model, current_gene)).reactions]

def pwy_dict_to_df(pwy_dict):
    df = pd.DataFrame.from_dict({k: ' + '.join(v) for k, v in pwy_dict.items()},
#     df = pd.DataFrame.from_dict({k: '+'.join(v) for k, v in pwy_dict.items()},
                       orient='index', columns=['Pathway'])
    df.index.name = 'Gene_inhibition'
    return df

def convert_dict_to_long_df(d: dict, cols: list):
    gcomb_pwy_list = list()
    for gcomb, pwys in d.items():
        for pwy in pwys:
            gcomb_pwy_list.append([gcomb, pwy])
#     index_col = [col for col in cols if col in 'Gene_inhibition|Reaction']
    df = pd.DataFrame.from_records(gcomb_pwy_list, columns=cols)
    return df

def get_SG_pwy_dict(model):
    gene_rct_dict = defaultdict(list)
    for current_gene in alpha_table.index:
        gene_rct_dict[f'{current_gene}'] = record_rct(model, current_gene)
    
    gene_pwy_dict = defaultdict(list)
    for current_gene,rcts in gene_rct_dict.items():
        for current_reaction in rcts:
            try:
                label = label_df.loc[f'{current_reaction}', 'm_subsystem']
                label = label if type(label) == str else label[0]
                gene_pwy_dict[f'{current_gene}'].append(label)
            except:
                print(current_gene, f'reaction {current_reaction} not exist in list')
        gene_pwy_dict[f'{current_gene}'] = set(convert_arg_to_list(gene_pwy_dict[f'{current_gene}'])) | set(gene_pwy_dict[f'{current_gene}'])
    return gene_pwy_dict

def get_gene_pathway_df(model):
    pwy_dict = get_SG_pwy_dict(model)
    DG_pwy_dict = get_DG_pwy_dict(pwy_dict)
    
    pwy_dict.update(DG_pwy_dict)
    
    gene_pathway_df = convert_dict_to_long_df(pwy_dict, ['Gene_inhibition', 'Pathway'])
    gene_pathway_df['XG'] = gene_pathway_df.Gene_inhibition.str.split('.').apply(
        lambda x: 'SG' if len(x)==1 else 'DG') 
    return gene_pathway_df

def antagonistic_count(genes:list):
    antagonistic_set = ['dadX.rffG', 'dadX.pyrD', 'acnB.thrB', 'acnB.thrB', 'acnB.thrB',
       'aroA.argD', 'aroA.argD', 'aroA.argD', 'dadX.guaB', 'aroA.dapB',
       'aroA.guaB']
    return list((set(genes).intersection(antagonistic_set)))

def get_ratio_col(gene_count_df):
    DG = gene_count_df.query('XG=="DG"').reset_index(drop=True).copy()
    DG['Antagonistic_gcomb'] = DG.Gene_list.apply(lambda x: antagonistic_count(x.split(', ')))
#     DG['Antagonistic_ratio'] = DG.Antagonistic_gcomb.str.split(', ').map(len)
    DG['Antagonistic_ratio'] = pd.Series(map(len, DG.Antagonistic_gcomb.tolist()))/DG.Gene_count
    DG['Antagonistic_gcomb'] = DG['Antagonistic_gcomb'].apply(lambda x: ', '.join(x))
    
    return DG[['Pathway', 'Antagonistic_ratio', 'Antagonistic_gcomb']]

def get_gene_count_df(gene_pathway_df):

    gene_count_df = (gene_pathway_df.groupby(['Pathway','XG'], as_index=False).agg(
                        Gene_list=('Gene_inhibition', lambda x: ', '.join(x.values)),
                        Gene_count=('Gene_inhibition', 'count')))
    gene_count_df = gene_count_df.merge(get_ratio_col(gene_count_df), on='Pathway')
    (gene_count_df.pivot(index=['Pathway','Antagonistic_ratio', 'Antagonistic_gcomb'], columns=['XG'], values=['Gene_list','Gene_count'])
         .sort_values(by=('Gene_count','DG'), ascending=False)).to_csv('./Data/gene_count.csv')
    return gene_count_df

def get_single_pathway_df(gene_pathway_df):
    # add p_o afterwards
    df = (gene_pathway_df.groupby(['Gene_inhibition','XG'], as_index=False).agg(
        Pathway_count=('Pathway', 'count'),
        Pathway_list = ('Pathway', lambda x: list(x.values))))
#         P = ('Pathway', lambda x: pd.DataFrame(x.values))))
    df['Single_pathway'] = pd.cut(df['Pathway_count'], [0,1,10], labels = ['Single_pathway', 'Multi_pathway'])
    return df.sort_values(['Pathway_count', 'XG'], ascending=True).set_index('Gene_inhibition')


In [ ]:
def get_rct_pathway_df():
    pwy_rct_dict = defaultdict(list)
    for index, sub_df in label_df[['m_subsystem']].iterrows():
        pwy_rct_dict[sub_df[0]].append(index)
    rct_pathway_df = convert_dict_to_long_df(pwy_rct_dict,['Pathway','Reaction'])
    return rct_pathway_df.drop_duplicates()

In [ ]:
def get_DG_pwy_dict(SG_pwy_dict):
    comb_list = list(pd.read_csv('./Data/gr_Div_DG_Blis_Aug31.csv').gene_inhibition[1:])
    DG_pwy_dict = dict()
    # for DG_pwy in comb_list:
    gene_pathway_row = list()
    for DG_pwy in comb_list:
        gene_pair = DG_pwy.split('.')
        DG_pwy_dict[DG_pwy] = SG_pwy_dict[gene_pair[0]] | SG_pwy_dict[gene_pair[1]] 
    return DG_pwy_dict

In [ ]:
rct_pathway_df = get_rct_pathway_df()
gene_pathway_df = get_gene_pathway_df(E0)

gene_count_df=get_gene_count_df(gene_pathway_df)
single_pathway_df = get_single_pathway_df(gene_pathway_df)

# p_o

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
def get_syn_df(gr_XG, model):
    model_id = model.id
    additive_threshold = .01
    diff_bins = [-10,-1*additive_threshold,1*additive_threshold,10]
    gr_bins = [-1,.2,1,10]
    
    if gr_XG.index.name == 'gene_inhibition':
        gr_XG.index.name = 'Gene_inhibition'
    syn_df  = pd.DataFrame([], index=gr_DG.index)
    syn_df['Predicted_growth_rate'] = gr_XG[f'Predicted_additive_effect_{model.id}_coculture']
    syn_df['Observed_growth_rate'] = gr_XG[f'{model.id}_coculture']
    syn_df['P_O'] = syn_df.Predicted_growth_rate - syn_df.Observed_growth_rate
    syn_df['Drug_comb_effect'] = pd.cut(syn_df['P_O'], bins=diff_bins, labels=['Antagonistic', 'Additive', 'Synergistic'])
    
    syn_df['PGR_bin'] = pd.cut(syn_df['Predicted_growth_rate'], bins=gr_bins, labels=['Low', 'Normal', 'High'])
    syn_df['OGR_bin'] = pd.cut(syn_df['Observed_growth_rate'], bins=gr_bins, labels=['Low', 'Normal', 'High'])
    
    syn_df.loc[(syn_df.Predicted_growth_rate < 1.5e-8) & (syn_df.Observed_growth_rate < 1.5e-8),'P_O'] = 0
    syn_df['gene_sort'] = (syn_df['P_O'] < 0)*10 + abs(syn_df['P_O'])
    syn_df['Species'] = model.id
    return syn_df

def get_antagonistic_df(syn_df):
    antagonistic_list = syn_df.loc[syn_df['P_O']<0].query("P_O<-0.01").index 
    # ? func get pwy col
    potential_pwy = (gene_combo_pathway
                                      .loc[antagonistic_list,'Pathway']
                                      .str.split(' \+ ') # series string need \+
                                      .apply(lambda x: sorted(x))) 
    return pd.DataFrame(potential_pwy)

def remove_nan_from(x: pd.Series):
    return x.apply(lambda x: sorted(list(itertools.compress(x,[ele not in [None, np.nan] for ele in x]))))
  

In [ ]:
def get_p_o_df():
    single_pathway_df = pd.read_csv('./Data/single_pathway_df.csv')
    gcomb_single_pathway_df = single_pathway_df.query('XG=="DG"') # select only DG
    p_o = pd.concat([get_syn_df(gr_DG, E0), get_syn_df(gr_DG, S0)])
    p_o = p_o.merge(gcomb_single_pathway_df.drop('XG', axis=1), left_index=True, right_on='Gene_inhibition', how='outer')
    return p_o.set_index('Gene_inhibition')

In [ ]:
p_o_full = get_p_o_df()

# flux

In [ ]:
def get_gene_id(model, gene_name):
    for i in model.genes:
        if(i.name == gene_name):
            return(i.id)

def get_rcts_list(model, gcomb_list): 
    rcts_list = list()
    rcts_set = set()
    for i, gene in enumerate(gcomb_list):
        gene_rcts = [rct.id for rct in model.genes.get_by_id(get_gene_id(model, gene)).reactions]
        if i > 0:
            gene_rcts = list(set(gene_rcts) - rcts_set)
        rcts_list.append(gene_rcts)
        rcts_set = rcts_set | set(gene_rcts)
    return rcts_list

def adjust_flux_df(model, df, gene_combo: list):
    if 'Normal' not in gene_combo:
#         gene_combo_dict = get_gcomb_alpha_dict(gene_combo) 
        v1_cols = df.filter(regex='v1').columns
        orig_cols = [ele.replace("_v1", '') for ele in df.filter(regex='v1').columns]
        
        gene_combo_list = gene_combo.split('.')
        rcts_list = get_rcts_list(model, gene_combo_list)
        scaled_rcts = list()
        for gene, rcts in zip(gene_combo_list, rcts_list):  
            rcts = [rct for rct in rcts if rct in orig_cols]
            for orig_col in rcts:
                alpha = alpha_table.loc[f'{gene}', f'{model.id}']   
                v1_col = orig_col + "_v1"
                df[f'{orig_col}'] = (df[f'{orig_col}'] + df[f'{v1_col}'])/alpha # only forward or backward != 0               
                df = df.drop(f'{v1_col}', axis=1) 
#     print(df.filter(regex='v1').columns)
    return df

def get_desired_BM(Biomass_df = Biomass_df, regex='Normal_coculture'):
    return Biomass_df.filter(regex=regex)
  
def supply_line(file):
#     file = f'./Data/{file}'
    with open(file) as fh:
        for line in fh:
            yield json.loads(line) 

def retrive_specific_culture(file_list: list, culture_item: str, XG = 'DG'): # culture_item: 'coculture_flux'
    def modify_culture_dict_keys(k: str, XG: str):
        k_list = k.split('_')
        k_list[:-2] =  ['.'.join(k_list[:-2])]
        k_list.append(XG)
        return '_'.join(k_list)
#     culture_item = 'E0_coculture_flux'
    flux_dict = dict()
    culture_dict = dict()
    
    for file, current_XG in zip(convert_arg_to_list(file_list), convert_arg_to_list(XG)):
        supp = supply_line(file)
        culture_items = [f'E0_{culture_item}', f'S0_ac_{culture_item}'] # prefix in json 
        for line in supp:    # culture as column <=> keys
    #         culture_dict = line.get(f'{culture_item}')
            if any([ele in line.keys() for ele in culture_items]):
                culture_dict.update(
                    {modify_culture_dict_keys(k, current_XG): v 
                     for k, v in line.items() if k in culture_items}) 
    return culture_dict # {E0_coculture: Dict, S0_coculture: Dict}

# pwy group with more than 2 element
def get_pwypair_gp_GRT2(p_o: dict): # index greater than 2 element
    flux_compare_combos_dict = {k:v for k,v in p_o.items() if len(v)>=2}
    return flux_compare_combos_dict
 
def retrive_specific_keys(model, file: List[str]|str, culture_item, desired_cycle):
    def get_flux_dict(culture_dict, model, desired_cycle):
#         if model.id in culture_dict.keys(): # ?delete?
#             culture_dict = culture_dict[model.id]
        flux_dict, no_grow = dict(), list()
        culture = culture_item.split('_')[0]
#         for current_gene_combo, cycle in desired_cycle[[f'{model.id}_{culture}']].iterrows():
        for current_gene_combo, cycle_row in desired_cycle.query('Species == @model.id & culture == @culture').iterrows():
#         for current_gene_combo, cycle in desired_cycle.iterrows():
            
#             cycle = cycle[0] # iterrow create subdata frame of one column 
            cycle = cycle_row['cycle_max_gr'] # iterrow create subdata frame of one column 
            if cycle < 15:
                no_grow.append(current_gene_combo)
                flux_dict[current_gene_combo] = pd.DataFrame([], index = [f'{current_gene_combo}'])
            else:
                desired_js = culture_dict.get(f'{current_gene_combo}')
                temp_df = pd.read_json(desired_js).query("cycle == @cycle")
                temp_df.index = [current_gene_combo]
        #                 return temp_df, current_gene_combo
                flux_dict[current_gene_combo] = adjust_flux_df(model, temp_df, current_gene_combo) # df as 
        #             genes_dict = {current_gene_combo: genes_dict.get(f'{current_gene_combo}') for current_gene_combo in genes_dict}  #subset of gene
        if no_grow:
            print('Zero growth: ', ', '.join(no_grow))
        return pd.concat(flux_dict.values()).copy() # pd.read_json() culture_item -> col pd.read_json()
#         return pd.concat(flux_dict.values()) # pd.read_json() culture_item -> col pd.read_json()
    
#     print(type(file) is dict)
    if type(file) is dict:
        return get_flux_dict(file, model, desired_cycle)
    else: # list of filename, str filename
        culture_dict = retrive_specific_culture(file, culture_item)
        return get_flux_dict(culture_dict, model, desired_cycle) 
    
def remove_Zero_col(df): # extend N differ than 0 
    return(df.loc[:, ((df !=0) & (df.notnull())).any(axis=0)]) # ignore NA entry 

def clean_flux_df(df):
    return remove_Zero_col(df.dropna())

def get_XG_cycle_from(desired_cycle):
    SG_cycle = desired_cycle.loc[[len(ele.split('.')) ==1 for ele in desired_cycle.index]]
    DG_cycle = desired_cycle.loc[[len(ele.split('.')) >=2 for ele in desired_cycle.index]]
    SG_cycle.index.name='SG'
    DG_cycle.index.name='DG'
    SG_cycle.columns.name=None
    DG_cycle.columns.name=None    
    return SG_cycle, DG_cycle    
alpha_table.columns = ['E0', 'S0.ac', 'S0.glc']

In [ ]:
def get_SG_DG(explode=True):
    df = pd.DataFrame(pd.DataFrame(2*[DG_list], index=['Gene_inhibition','SG']).T.set_index('Gene_inhibition').SG.str.split('.'))
    return df.explode() if explode else df
   
def get_alpha_wide():
    return alpha_table.melt(value_vars=['E0', 'S0.ac'], var_name='Species', value_name='alpha', ignore_index=False)

def get_alpha_to_merge():
    def get_alpha(l: list[str, str]):
        l=l[0]
    #     print(l)
        Species = [E0.id, S0.id]
        df = pd.DataFrame(alpha_table.loc[l, Species]).T
        return df.values.tolist()
    # a.apply(get_alpha, axis=1, result_type='broadcast')

    ES_cols = ['E0', 'S0.ac']
    alpha_wide = get_alpha_wide()
    df = (get_SG_DG(explode=False)
          .apply(lambda x: pd.Series(get_alpha(x), index=ES_cols), axis=1) # df with E0 S0 alpha columns
          .melt(value_vars=ES_cols, var_name='Species', value_name='alpha', ignore_index=False)
          .combine_first(alpha_wide))
    return df
alpha_to_merge = get_alpha_to_merge()

In [ ]:
def set_GI_SP_as_MI(end_BM, manual_SI = False):
    if type(end_BM.index[0]) is not int: # Gene_inhibition is not index # Series.dtype -> dtype('int64')
        if end_BM.index.name in [None, 'DG']:
            end_BM.index.name = 'Gene_inhibition'
        if end_BM.index.name == 'Gene_inhibition':
            end_BM = end_BM.reset_index()
    if 'Species' in end_BM.columns and manual_SI == False:
        return end_BM.set_index(['Gene_inhibition', 'Species'])
    else:
        return end_BM.set_index('Gene_inhibition')

def join_dfs_using_MI(df1, df2, how='left'):
    return set_GI_SP_as_MI(df1).join(set_GI_SP_as_MI(df2), how=how)


In [ ]:
def get_flux_compare_df(culture_item,culture_dict, SG_cycle, DG_cycle):
    file_list = ['./Data/flanalysis_BM_SG.json', './Data/flanalysis_BM_DG.json'] 
    culture_dict = retrive_specific_culture(file_list, culture_item=culture_item, XG=['SG','DG'])  
    flux_dict = dict()
    for species, XG_cycle in itertools.product([E0, S0],[DG_cycle, SG_cycle]):
        current_XG = XG_cycle.index.name
        current_species = species.id
        print(current_species, current_XG)

        key = '_'.join([current_species, culture_item, current_XG])

        flux_df = retrive_specific_keys(species, culture_dict[key], culture_item, XG_cycle)
        flux_df['Species'] = current_species
        flux_df['XG'] = current_XG
        flux_dict[(current_species, current_XG)] = flux_df
    flux_compare_df = pd.concat(flux_dict.values()).copy()
    flux_compare_df.index.name = 'Gene_inhibition'
#     flux_compare_df = get_ESdiff(remove_Zero_col(flux_compare_df)) # ES diff 41 columns ?only common
#     flux_compare_df = get_Ndiff(flux_compare_df)
    flux_compare_df = join_dfs_using_MI(get_alpha_to_merge(), flux_compare_df, how='right') # DG only
    return flux_compare_df

In [ ]:
file_list = ['./Data/flanalysis_BM_SG.json', './Data/flanalysis_BM_DG.json']
culture_dict = retrive_specific_culture(file_list, culture_item='coculture_flux', XG=['SG','DG'])
SG_cycle, DG_cycle = get_XG_cycle_from(desired_cycle)
# flux_compare_df = get_flux_compare_df('coculture_flux', culture_dict, SG_cycle, DG_cycle) 
flux_compare_df = pd.read_csv('./Data/flux_compare_df.csv')
# flux_compare_df.to_csv('./Data/flux_compare_df.csv')

# Reaction column

In [ ]:
def get_SG_pwy_df():
    SG_pwy_df = pd.DataFrame.from_dict(get_SG_pwy_dict(E0), orient='index')
    SG_pwy_df['Pathways'] = SG_pwy_df.values.tolist()
    SG_pwy_df = pd.DataFrame(remove_nan_from(SG_pwy_df.Pathways))
#     SG_pwy_df = pd.DataFrame(SG_pwy_df.Pathways
#                              .apply(lambda x: remove_nan_from(x)))
    SG_pwy_df['E0_Reactions'] = list(pd.Series(SG_pwy_df.index).apply(lambda x: record_rct(E0,x)))
    SG_pwy_df['S0_Reactions'] = list(pd.Series(SG_pwy_df.index).apply(lambda x: record_rct(S0,x)))    
    return SG_pwy_df

def find_reactions_not_exist_in(model, reactions: list):
    if len(reactions) ==0:
        return reactions
    reactions = convert_arg_to_list(reactions)
    reactions_all = [rct.id for rct in model.reactions]
    not_found_reactions = [rct for rct in reactions if rct not in reactions_all]
    return not_found_reactions

def get_pwy_rct_df(DG_list):
    def get_pwys_from(SG_pwy_df, SG):
        return SG_pwy_df.loc[SG, 'Pathways']
    
    g_pwy_rct_df = pd.DataFrame(DG_list, columns=['DG'])
    g_pwy_rct_df['SG_list'] =  g_pwy_rct_df.apply(lambda x: x.DG.split('.'), axis=1)
    g_pwy_rct_df['Pathways'] = g_pwy_rct_df.apply(lambda x: list(set(SG_pwy_df.loc[x.SG_list[0],'Pathways']) | set(SG_pwy_df.loc[x.SG_list[1],'Pathways'])), axis=1)
    g_pwy_rct_df['Common_pathway'] = g_pwy_rct_df.apply(lambda x: set(get_pwys_from(SG_pwy_df, x.SG_list[0]))
                                                        .intersection(set(get_pwys_from(SG_pwy_df, x.SG_list[1])))
                                                        ,axis=1)
    g_pwy_rct_df['Reactions_E0'] = g_pwy_rct_df.apply(
        lambda x: list(set(SG_pwy_df.loc[x.SG_list[0],'E0_Reactions']) | set(SG_pwy_df.loc[x.SG_list[1],'E0_Reactions'])), axis=1)
    g_pwy_rct_df['Reactions_S0'] = g_pwy_rct_df.apply(
        lambda x: list(set(SG_pwy_df.loc[x.SG_list[0],'S0_Reactions']) | set(SG_pwy_df.loc[x.SG_list[1],'S0_Reactions'])), axis=1)
    g_pwy_rct_df['Common_Reactions'] = g_pwy_rct_df.apply(lambda x: set(x['Reactions_E0']).intersection(set(x['Reactions_S0'])), axis=1)
    g_pwy_rct_df['Reactions_E0_only'] = g_pwy_rct_df.apply(lambda x: set(x['Reactions_E0'])-set(x['Reactions_S0']), axis=1)
    g_pwy_rct_df['Reactions_S0_only'] = g_pwy_rct_df.apply(lambda x: set(x['Reactions_S0'])-set(x['Reactions_E0']), axis=1)
    g_pwy_rct_df['Reactions_not_in_S0'] = g_pwy_rct_df.apply(lambda x: find_reactions_not_exist_in(S0, x.Reactions_E0_only), axis=1)
    g_pwy_rct_df['Reactions_not_in_E0'] = g_pwy_rct_df.apply(lambda x: find_reactions_not_exist_in(E0, x.Reactions_S0_only), axis=1)
    return g_pwy_rct_df.set_index('DG')

In [ ]:
SG_pwy_df = get_SG_pwy_df()
DG_list = [ele for ele in set(desired_cycle.index) if len(ele.split('.')) >=2]
pwy_rct_df = get_pwy_rct_df(DG_list)

# BM to merge

In [ ]:
def get_BM_bin(BM): # separate S0 E0 bin, combine again
    sub_BM = BM.pivot(columns='Species', values='Biomass') #merge gene and species
    sub_BM['S0_BM_bin'] =  pd.cut(sub_BM['S0.ac'], [-1,.001,.0025,1], labels = ['Low', 'Medium','High'])
    sub_BM['E0_BM_bin'] =  pd.cut(sub_BM['E0'], [-1,.001,.0065,1], labels = ['Low', 'Medium','High'])
    return sub_BM.melt(value_vars=['S0_BM_bin','S0_BM_bin'],value_name='BM_bin',ignore_index=False).BM_bin


In [ ]:
def get_end_BM():
    BM = pd.DataFrame(Biomass_df.loc[:, [col for col in Biomass_df.columns if 'coculture' in col]].iloc[-1]
                     ).rename(columns={350:'Biomass'})
    BM['Species'] = list(pd.Series(BM.index).apply(lambda x: str(x).split('_')[0]))
    BM['Gene_inhibition'] = list(pd.Series(BM.index).apply(lambda x: str(x).split('_')[1]))
    BM = BM.set_index(['Gene_inhibition'])
    BM['Total_BM'] = BM.groupby('Gene_inhibition').Biomass.sum()

    BM['Total_BM_bin'] = pd.cut(BM['Total_BM'], [-1,.004,.008,1], labels = ['Low', 'Medium','High'])
    BM['BM_bin'] = get_BM_bin(BM)
    return BM

def separate_Species_df(df, model, inc_Species = False):
    df = df.loc[:,~df.columns.str.contains('Ndiff|ESdiff')]
    temp_df = pd.concat([df['Species'],df.select_dtypes(include = ['float'])], axis=1)
    def get_species_loc(model):
        return list(temp_df.Species == model.id)
    result_df = temp_df.loc[get_species_loc(model)]
    return result_df if inc_Species else result_df.drop('Species',axis=1)

def add_to_end_BM():
    def get_ratioNstd_col(model):
        temp_df = separate_Species_df(end_BM, model, inc_Species=True)
#         temp_df[f'{model.id}_BM_ratio'] = temp_df.Biomass/temp_df.Total_BM
#         temp_df[f'{model.id}_standardized_BM'] = temp_df.Biomass/temp_df.loc['Normal', 'Biomass']
        temp_df[f'BM_consortia_frac'] = temp_df.Biomass/temp_df.Total_BM
        temp_df[f'standardized_BM'] = temp_df.Biomass/temp_df.loc['Normal', 'Biomass']
        return temp_df
    return pd.concat([get_ratioNstd_col(E0), get_ratioNstd_col(S0)]).set_index('Species', append=True).drop(['Total_BM', 'Biomass'], axis=1)

end_BM = get_end_BM()
end_BM = set_GI_SP_as_MI(add_to_end_BM()).join(set_GI_SP_as_MI(end_BM))

# Merge all

In [ ]:
desired_cycle

In [ ]:
join_dfs_using_MI(merged_df, next_df, how='left')

In [ ]:
flux_compare_df

In [ ]:
def get_full_df(): # Missing desired_cycle
    df_list = [p_o_full, end_BM, pwy_rct_df, flux_compare_df]
    merged_df = df_list.pop(0)
    for i, next_df in enumerate(df_list):
        manual_SI = False
        if 'Species' not in next_df:
            manual_SI = True
        merged_df = (set_GI_SP_as_MI(merged_df)
                     .join(set_GI_SP_as_MI(next_df), how='left')) # left join->DG only
        # inner_join SG info from desired cycle & alpha & flux
    return merged_df.reset_index(level='Species')

full_df = get_full_df() # full_df.loc[:,'Predicted_growth_rate':'Reactions_not_in_E0']


# query spec reaction flux

In [ ]:
# calculate flux difference between Normal and between E&S
def get_Ndiff(df, merged = True):
    result_df = pd.concat([separate_Species_df(df, E0, inc_Species=True), separate_Species_df(df, S0, inc_Species=True)])
    return set_GI_SP_as_MI(df).join(set_GI_SP_as_MI(result_df), rsuffix='_Ndiff') if merged else result_df
def get_ESdiff(df, merged = True):
    ESdiff = separate_Species_df(df, E0) - separate_Species_df(df, S0)
    return df.merge(ES_flux_diff, left_index=True, right_index=True, suffixes=[None, '_ESdiff']) if merged else ESdiff

def get_main_role(df, colname='Normal'):
    return pd.DataFrame(df.apply(lambda x: 'Equal' if x.max()/1.3 < x.min() else x.idxmax()), columns=convert_arg_to_list(colname))
Normal_main_role = full_df.query("index=='Normal'").set_index('Species').select_dtypes(include = ['float']).dropna(axis=1)
Normal_main_role = get_main_role(Normal_main_role)

def classify_main_role_diff(df):
    if df.loc['Query'] > df.loc['Normal']:
        return 'E0'  
    if df.loc['Query']== df.loc['Normal']:
        return 'Role not change'
    return 'S0'

def add_main_role_diff_cols(gene_list, query_list_rct=None): # df = add_main_role_diff_cols(DG_list)
    out_dict = {'Normal': [[],[]]}
    for current_gene in convert_arg_to_list(gene_list):
        check_main_role = get_query_sub_df(full_df, current_gene, query_list_rct=query_list_rct, RCT_only=True, inc_Normal=False).set_index('Species').dropna() # same to higher
        main_role_diff = (get_main_role(check_main_role, 'Query')
                          .join(Normal_main_role)
#                           .apply(lambda x: (x.loc['Query']!=x.loc['Normal']) & (x.loc['Query']!='Equal') & (x.loc['Normal']!='Equal'),axis=1))
                          .apply(lambda x: classify_main_role_diff(x), axis=1))
        main_role_diff = main_role_diff[lambda x: x != 'Role not change']
        diff_list = list(main_role_diff.index) # Differ index
#         diff_count = sum(main_role_diff)
#         rct_len = len(main_role_diff)
#         GI_switch_main_role = diff_count/rct_len
        diff_how = list(main_role_diff)
        out_dict[current_gene] = [diff_list, diff_how]
#         out_dict[current_gene] = [diff_list, diff_count, rct_len, GI_switch_main_role]
#     result_df = pd.DataFrame([diff_list, [diff_count]], columns=current_gene,index=['GI_diff_rct', 'GI_diff_count']).T
#     result_df = pd.DataFrame.from_dict(out_dict, orient='index', columns=['GI_diff_rct', 'GI_diff_count', 'GI_rct_count', 'GI_percent_switched'])
    result_df = pd.DataFrame.from_dict(out_dict, orient='index', columns=['GI_diff_rct', 'GI_whos_role_increased'])
    result_df.index.name = 'Gene_inhibition'
    return result_df

def get_diff_analysis_df(gene_list, query_list_rct=None):
    df = add_main_role_diff_cols(gene_list, query_list_rct)
    df = (df.join(full_df[[ # np.array(full_df.loc[:,:'Reactions_not_in_E0'].columns)
        'Species','BM_consortia_frac', 'standardized_BM','Total_BM','Total_BM_bin','Drug_comb_effect',
        'Observed_growth_rate', 'Pathways']]))
    df['group'] = df.groupby('Gene_inhibition').ngroup()
    df = df.sort_values(['Drug_comb_effect','group']).drop('group', axis=1)
#     df.to_csv('test.csv')
    return df
# diff_analysis_df = get_diff_analysis_df()

def get_full_cols(upto = 'Reactions_not_in_E0'):
    return np.array(full_df.loc[:,:upto].columns)

# CTC cycle reaction 

In [ ]:
rct_pathway_df = pd.read_csv('./Data/rct_pathway.csv')

In [ ]:
TAC_rct_list = rct_pathway_df.query('Pathway == "Citric Acid Cycle"').Reaction.unique()

In [ ]:
TAC_rct_list

In [ ]:
def get_query_sub_df(full_df, query_list, query_list_rct=None, RCT_only=False, inc_Normal=True): # ['dadX.mrdA', 'dadX.pyrD']
    if type(full_df.index) is pd.MultiIndex: # free one level
        full_df = full_df.reset_index(level='Species')
    query_list = convert_arg_to_list(query_list)
    if query_list_rct is None:
        query_list_rct = list()
        for query in query_list:
            query_list_rct.append(list(set(pwy_rct_df.loc[query, 'Reactions_E0']).intersection(set(full_df.columns)))) # pwy_rct_df specific to  E0 pathway
        query_list_rct = list(set(itertools.chain(*query_list_rct)))
    else:
        query_list_rct = [rct for rct in TAC_rct_list if rct in full_df.columns]

#     desired_cols = full_df.loc[:,'Predicted_growth_rate':'Reactions_not_in_E0'] if not RCT_only else pd.DataFrame([])
    desired_cols = full_df.loc[:,:'Reactions_not_in_E0'] if not RCT_only else full_df[['Species']]
    desired_cols = list(itertools.chain(*[desired_cols, query_list_rct]))
#     result_df = pd.join([result_df, full_df.loc[list(itertools.chain(*[['Normal'],query_list])), query_list_rct]])
    if inc_Normal:
        query_list = itertools.chain(*[['Normal'], query_list])
    return full_df.loc[query_list,desired_cols]
# get_query_sub_df(full_df, DG_list, )


# TCA_diff_df = get_diff_analysis_df(DG_list, TAC_rct_list)
TCA_diff_df = add_main_role_diff_cols(DG_list, TAC_rct_list)


In [ ]:
a = get_diff_analysis_df(DG_list, TAC_rct_list)
a.sort_values('Total_BM_bin').to_csv('test.csv')

# plot coculture

In [ ]:
def plot_coculture(gene_list):
    colnames = list()
    for model, current_gene in itertools.product([E0, S0],gene_list):
        colnames.append('_'.join([model.id, current_gene, 'coculture']))
    
    Biomass_df[colnames].plot()
    return Biomass_df[colnames]
# plot_coculture(['dadX.pyrD'])

In [1]:
kl

NameError: name 'kl' is not defined